**Ejemplo de cálculo de la norma $2$ al cuadrado de un vector**

In [1]:
%%file norm_square.py
n=10**7
vector=list(range(n))
norm=0
for v in vector:
    norm+=v*v

Overwriting norm_square.py


In [2]:
%%bash
echo "-1" |sudo tee -a /proc/sys/kernel/perf_event_paranoid

-1


In [3]:
%%bash
perf stat -S -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square.py


 Performance counter stats for 'python3 norm_square.py' (20 runs):

        5123821017      cycles                                                        ( +-  0.55% )  (74.95%)
       14073511390      instructions              #    2.75  insn per cycle           ( +-  0.69% )  (75.01%)
          42426825      cache-references                                              ( +-  0.20% )  (75.06%)
          19997224      cache-misses              #   47.133 % of all cache refs      ( +-  0.13% )  (74.98%)

            1.9227 +- 0.0113 seconds time elapsed  ( +-  0.59% )



In [6]:
#%%bash
#perf stat -A --all-cpus -S -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square.py

**Comentarios:**

* Con `perf` se repiten las mediciones utilizando la *flag* `-r`. Con la *flag* `-e` se enlistan las métricas a calcular. `-S` llama a [sync](http://man7.org/linux/man-pages/man2/sync.2.html) antes de iniciar la ejecución del programa. `-A` no agregar los conteos a lo largo de los *cores* monitoreados. Ver `perf stat --help`.

* En el ejemplo anterior además de los ciclos e instrucciones se calculan los *cache references* y los *cache misses*. Ver [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb) para definiciones de *cache references* y *misses*. Esencialmente el sistema de memoria buscó el número que aparece en *cache-references* por datos o instrucciones en el caché y de éstas búsquedas, en *cache-misses*, se repora el número (y porcentaje) de búsquedas fallidas que no estuvieron en memoria. 

* Un factor que incrementa el número de *cache-misses* es la fragmentación de datos, ver [Fragmentation](https://en.wikipedia.org/wiki/Fragmentation_(computing)). La fragmentación incrementa el número de transferencias de memoria hacia la CPU y además imposibilita la vectorización porque el caché no está lleno. El caché puede llenarse sólo al tener bloques contiguos de memoria alojados para los datos (recuérdese que la interconexión o *bus* sólo puede mover *chunks* de memoria contigua, ver [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb)). Python en su implementación común CPython, tiene **data fragmentation** por ejemplo al usar listas. Las listas de Python alojan locaciones donde se pueden encontrar los datos y no los datos en sí. Al utilizar tales estructuras para operaciones matriciales el *runtime* de Python debe realizar *lookups* por índices y al no encontrarse de forma contigua los datos, el *overhead* en la transferencia de datos es mayor lo que causa que se tengan mayor número de *cache-misses* y que los cores tengan que esperar hasta que los datos estén disponibles en el caché.

* Obsérvese que se reporta el IPC al lado de *instructions* con nombre *insn per cycle*. Un alto IPC indica una alta transferencia de instrucciones de la unidad de memoria hacia la unidad de cómputo y un menor IPC indica más **stall cycles**, ver [pipeline stall](https://en.wikipedia.org/wiki/Pipeline_stall). Sin embargo como se mencionó antes, se debe evaluar si *a high rate of instructions* indica un *high rate of actual work completed* (p.ej. un *loop* tiene una alta tasa de IPC pero no siempre se realiza trabajo útil). Ver sección *CPU statistics* en la [liga](http://www.brendangregg.com/perf.html). `perf` también tiene una métrica para medir el número de ciclos que se utilizaron para esperar a ejecutar instrucciones (los cores están *stalled*). Las métricas son: `stalled-cycles-frontend` y `stalled-cycles-backend`.

También podemos obtener las estadísticas por core. Obsérvese que la salida anterior de `perf` calculó el IPC a partir del número de instrucciones y ciclos ahí reportados. El mismo cálculo se realiza a continuación:

In [7]:
#%%bash
#perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square.py

In [8]:
%%bash
perf stat -S --all-cpus -A -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square.py


 Performance counter stats for 'system wide' (20 runs):

CPU0               298120352      cycles                                                        (75.11%)
CPU1               301648607      cycles                                                        (75.03%)
CPU2               310307744      cycles                                                        (75.03%)
CPU3               309422661      cycles                                                        (75.05%)
CPU4               290664122      cycles                                                        (75.11%)
CPU5               306978407      cycles                                                        (75.05%)
CPU6               300076931      cycles                                                        (75.07%)
CPU7               303777036      cycles                                                        (75.09%)
CPU8               314116473      cycles                                                        (75.11

Otras métricas pueden ser obtenidas si ejecutamos `perf` sólo con la *flag* `-r`:

In [9]:
%%bash
perf stat -S -r 20 python3 norm_square.py


 Performance counter stats for 'python3 norm_square.py' (20 runs):

           1927.51 msec task-clock                #    1.000 CPUs utilized            ( +-  0.62% )
                 1      context-switches          #    0.000 K/sec                    ( +- 20.20% )
                 0      cpu-migrations            #    0.000 K/sec                  
             99924      page-faults               #    0.052 M/sec                    ( +-  0.00% )
        5150266707      cycles                    #    2.672 GHz                      ( +-  0.58% )  (74.97%)
       14131213509      instructions              #    2.74  insn per cycle           ( +-  0.74% )  (75.01%)
        2929512564      branches                  # 1519.839 M/sec                    ( +-  0.85% )  (75.04%)
           1173976      branch-misses             #    0.04% of all branches          ( +-  0.68% )  (74.98%)

            1.9279 +- 0.0119 seconds time elapsed  ( +-  0.62% )



**Comentarios:**

* La métrica de *task-clock* indica cuántos *clock cycles* tomó nuestra tarea y se reporta en milisegundos. Obsérvese que también se indica cuántos CPU's fueron utilizados. En el *output* anterior no es exactamente 1 pues el programa no sólo involucra trabajo de CPU sino también de alojamiento de memoria. 

* *context-switches* y *cpu-migrations* indican cuánto se detuvo el programa para realizar:

    * operaciones relacionadas con el kernel del sistema (por ejemplo I/O).
    * ejecución de otras aplicaciones.
    * alojamiento de la ejecución en un core distinto (lo que ocasiona que haya movimiento de datos o instrucciones hacia el caché de otro core).
    
la idea es que nuestro programa tenga un número pequeño de éstas métricas.

* *page-faults* se relaciona con el alojamiento de memoria a partir del kernel del sistema operativo. Ocasiona que se detenga la ejecución del programa y por tanto deseamos que ésta métrica no sea muy grande. Ver [Page fault](https://en.wikipedia.org/wiki/Page_fault).

* Ver [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb) para el *branching*. Esencialmente interesa que el número de *branch misses* sea pequeño.



Podemos generar la salida anterior por *core*:

In [23]:
#%%bash
#perf stat -S -a --per-core -r 20 python3 norm_square.py

In [10]:
%%bash
perf stat -S --all-cpus -A -r 20 python3 norm_square.py


 Performance counter stats for 'system wide' (20 runs):

CPU0                 2004.52 msec cpu-clock                 #    1.001 CPUs utilized          
CPU1                 2004.52 msec cpu-clock                 #    1.001 CPUs utilized          
CPU2                 2004.96 msec cpu-clock                 #    1.001 CPUs utilized          
CPU3                 2004.99 msec cpu-clock                 #    1.001 CPUs utilized          
CPU4                 2004.97 msec cpu-clock                 #    1.001 CPUs utilized          
CPU5                 2004.94 msec cpu-clock                 #    1.001 CPUs utilized          
CPU6                 2004.91 msec cpu-clock                 #    1.001 CPUs utilized          
CPU7                 2004.88 msec cpu-clock                 #    1.001 CPUs utilized          
CPU8                 2004.85 msec cpu-clock                 #    1.001 CPUs utilized          
CPU9                 2004.81 msec cpu-clock                 #    1.001 CPUs utilized   

Para contrastar las salidas anteriores con un mejor uso del caché y de los *cores* se utiliza a continuación el paquete de `numpy`:

In [12]:
%%file norm_square_numpy.py
import numpy as np
n=10**7
vector=np.arange(n)
vector.dot(vector)

Overwriting norm_square_numpy.py


In [13]:
%%bash
perf stat -S -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'python3 norm_square_numpy.py' (20 runs):

       10883030366      cycles                                                        ( +-  1.92% )  (74.13%)
        3317792148      instructions              #    0.30  insn per cycle           ( +-  2.36% )  (75.34%)
          20753464      cache-references                                              ( +-  1.15% )  (75.80%)
           1743193      cache-misses              #    8.400 % of all cache refs      ( +-  7.94% )  (74.73%)

           0.23934 +- 0.00512 seconds time elapsed  ( +-  2.14% )



**Comentarios:** 

* Obsérvese que es **más tardado** este programa para el número $n$ que se está utilizando en comparación con el programa anterior sin vectorizar.

* El número de instrucciones es **menor** al reportado anteriormente y parece tener un mayor número $%$ de *cache-misses*. Obsérvese que para este ejemplo es más ilustrativo observar las estadísticas por *core* y realizar conclusiones:

In [21]:
#%%bash
#perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square_numpy.py

In [14]:
%%bash
perf stat -S --all-cpus -A -e cycles,instructions,cache-references,cache-misses -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'system wide' (20 runs):

CPU0                62508379      cycles                                                        (76.32%)
CPU1                26857115      cycles                                                        (76.26%)
CPU2                27978652      cycles                                                        (76.29%)
CPU3                24328931      cycles                                                        (76.22%)
CPU4                25400584      cycles                                                        (76.22%)
CPU5                23141653      cycles                                                        (76.12%)
CPU6                24341914      cycles                                                        (76.16%)
CPU7                22866352      cycles                                                        (76.25%)
CPU8                26054803      cycles                                                        (76.35

**Comentarios:** 

* Se observa que la métrica IPC está más uniforme a través de los cores lo que indica un mejor *load balancing*.

* Además, el $\%$ de *cache-misses* es menor que en el caso no vectorizado a diferencia del *output* anterior que indicaba un mayor $\%$ de *cache-misses*.

También podemos obtener las salidas anteriores de `perf` al usar `numpy`: 

In [16]:
%%bash
perf stat -S -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'python3 norm_square_numpy.py' (20 runs):

           4009.45 msec task-clock                #   17.208 CPUs utilized            ( +-  1.33% )
            564444      context-switches          #    0.141 M/sec                    ( +-  5.07% )
                50      cpu-migrations            #    0.013 K/sec                    ( +-  6.76% )
              4686      page-faults               #    0.001 M/sec                    ( +-  0.75% )
       10567628399      cycles                    #    2.636 GHz                      ( +-  1.28% )  (73.97%)
        3602865118      instructions              #    0.34  insn per cycle           ( +-  1.75% )  (75.34%)
         763664359      branches                  #  190.466 M/sec                    ( +-  1.77% )  (75.77%)
          15063370      branch-misses             #    1.97% of all branches          ( +-  1.82% )  (74.93%)

           0.23300 +- 0.00349 seconds time elapsed  ( +-  1.50% )



In [17]:
#%%bash
#perf stat -S -a --per-core -r 20 python3 norm_square_numpy.py

In [18]:
%%bash
perf stat -S --all-cpus -A -r 20 python3 norm_square_numpy.py


 Performance counter stats for 'system wide' (20 runs):

CPU0                  233.86 msec cpu-clock                 #    0.883 CPUs utilized          
CPU1                  233.84 msec cpu-clock                 #    0.883 CPUs utilized          
CPU2                  233.83 msec cpu-clock                 #    0.883 CPUs utilized          
CPU3                  233.85 msec cpu-clock                 #    0.883 CPUs utilized          
CPU4                  233.86 msec cpu-clock                 #    0.883 CPUs utilized          
CPU5                  233.87 msec cpu-clock                 #    0.883 CPUs utilized          
CPU6                  233.89 msec cpu-clock                 #    0.883 CPUs utilized          
CPU7                  233.87 msec cpu-clock                 #    0.883 CPUs utilized          
CPU8                  233.88 msec cpu-clock                 #    0.883 CPUs utilized          
CPU9                  233.89 msec cpu-clock                 #    0.883 CPUs utilized   

### Regresando al algoritmo *gaxpy column oriented*...

Utilizaremos `perf` en lo que sigue para evaluar las métricas de IPC, *cache-references*, *cache-misses* y medir el tiempo de ejecución. Además, esto permitirá comparar los tiempos de ejecución del algoritmo *gaxpy* (nivel BLAS 2), con vectorización y sin vectorización.

Consideramos una matriz $A \in \mathbb{R}^{10^4 \times 10^4}$ con entradas pseudoaleatorias. **No se sugiere ejecutar los siguientes ejemplos para máquinas que tengan menos de 8gb de memoria**:

In [20]:
import numpy as np

In [ ]:
np.random.seed(2020)
m=10**4
n=10**4
A=np.random.rand(m,n)
file='A.txt'
np.savetxt(file,A)

Arhivo sin vectorizar y utilizando listas para construir a la matriz y a los vectores:

In [8]:
%%file mult_matrix_vector.py
m=10**4
n=10**4
x=[2.5]*n
y=[0]*m
A = []
file='A.txt'
with open(file,'r') as f:
    for l in f:
        A.append([float(k) for k in l.replace('\n','').replace(' ',',').split(',')])      
for j in range(n):
    for i in range(m):
        y[i]+=A[i][j]*x[j]

Writing mult_matrix_vector.py


Archivo vectorizando con numpy:

In [9]:
%%file mult_matrix_vector_numpy.py
import numpy as np
m=10**4
n=10**4
x = 2.5*np.ones(n)
y = np.zeros(m)
file='A.txt'
A = np.loadtxt(file)
for j in np.arange(n):
    y+=A[:,j]*x[j]

Writing mult_matrix_vector_numpy.py


#### Caso no vectorizado

In [10]:
%%bash
sudo perf stat -S -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_vector.py


 Performance counter stats for 'python3 mult_matrix_vector.py' (2 runs):

      253636293168      cycles                                                        ( +-  2.57% )
      612211438956      instructions              #    2.41  insn per cycle           ( +-  0.80% )
        1021919811      cache-references                                              ( +-  0.04% )
         127117602      cache-misses              #   12.439 % of all cache refs      ( +-  0.56% )

      63.613271875 seconds time elapsed                                          ( +-  2.51% )



Métricas por core:

In [11]:
%%bash
sudo perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_vector.py


 Performance counter stats for 'system wide' (2 runs):

S0-C0           2          343119167      cycles                                                      
S0-C0           2           98370360      instructions              #    0.29  insn per cycle         
S0-C0           2            5840329      cache-references                                            
S0-C0           2            1414327      cache-misses              #   24.217 % of all cache refs    
S0-C1           2          528971112      cycles                                                      
S0-C1           2          258058789      instructions              #    0.49  insn per cycle         
S0-C1           2            8051433      cache-references                                            
S0-C1           2            2969543      cache-misses              #   36.882 % of all cache refs    
S0-C2           2        69421643702      cycles                                                      
S0-C2           

#### Caso vectorizando

In [12]:
%%bash
sudo perf stat -S -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_vector_numpy.py


 Performance counter stats for 'python3 mult_matrix_vector_numpy.py' (2 runs):

      243217588216      cycles                                                        ( +-  1.27% )
      598729874148      instructions              #    2.46  insn per cycle           ( +-  0.42% )
         662503038      cache-references                                              ( +-  1.39% )
          95202746      cache-misses              #   14.370 % of all cache refs      ( +-  0.97% )

      60.488697485 seconds time elapsed                                          ( +-  1.26% )



In [13]:
%%bash
sudo perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_vector_numpy.py


 Performance counter stats for 'system wide' (2 runs):

S0-C0           2         2664298731      cycles                                                      
S0-C0           2         1964872569      instructions              #    0.74  insn per cycle         
S0-C0           2           10837071      cache-references                                            
S0-C0           2            4435770      cache-misses              #   40.931 % of all cache refs    
S0-C1           2          964636936      cycles                                                      
S0-C1           2          320553130      instructions              #    0.33  insn per cycle         
S0-C1           2            6043601      cache-references                                            
S0-C1           2            2799592      cache-misses              #   46.323 % of all cache refs    
S0-C2           2         1339127149      cycles                                                      
S0-C2           

**Comentarios:**

* No se observa una gran diferencia entre el caso vectorizado y no vectorizado salvo que en el caso vectorizando se tiene un $\%$ mayor de *cache-misses*. Sería conveniente realizar más repeticiones de esta misma operación o con diferentes operaciones de nivel 2 BLAS para realizar conclusiones más sólidas. Ver [level 2](http://www.netlib.org/blas/#_level_2) para más ejemplos de algoritmos en el álgebra lineal en esta categoría.

* Otra opción que se tiene es utilizar la siguiente implementación ya que `numpy` soporta operaciones del tipo matriz-vector sin necesidad de utilizar un *for loop*:

In [14]:
%%file mult_matrix_vector_numpy_2.py
import numpy as np
m=10**4
n=10**4
x = 2.5*np.ones(n)
y = np.zeros(m)
file='A.txt'
A = np.loadtxt(file)
y=A@x

Writing mult_matrix_vector_numpy_2.py


In [15]:
%%bash
sudo perf stat -S -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_vector_numpy_2.py


 Performance counter stats for 'python3 mult_matrix_vector_numpy_2.py' (2 runs):

      243195411750      cycles                                                        ( +-  1.00% )
      602205576232      instructions              #    2.48  insn per cycle           ( +-  0.23% )
         553013970      cache-references                                              ( +-  0.65% )
          87666482      cache-misses              #   15.852 % of all cache refs      ( +-  0.47% )

      60.022411249 seconds time elapsed                                          ( +-  1.06% )



In [16]:
%%bash
sudo perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_vector_numpy_2.py


 Performance counter stats for 'system wide' (2 runs):

S0-C0           2         5768543148      cycles                                                      
S0-C0           2         2652993320      instructions              #    0.46  insn per cycle         
S0-C0           2           13669781      cache-references                                            
S0-C0           2            5700218      cache-misses              #   41.699 % of all cache refs    
S0-C1           2         1341384457      cycles                                                      
S0-C1           2          397736414      instructions              #    0.30  insn per cycle         
S0-C1           2            7366652      cache-references                                            
S0-C1           2            3922098      cache-misses              #   53.241 % of all cache refs    
S0-C2           2         1434196155      cycles                                                      
S0-C2           

**O con una versión *gaxpy row oriented***

In [5]:
%%file mult_matrix_vector_numpy_row_oriented.py
import numpy as np
m=10**4
n=10**4
x = 2.5*np.ones(n)
y = np.zeros(m)
file='A.txt'
A = np.loadtxt(file)
for i in np.arange(m):
    y[i]+=A[i,:].dot(x)

Overwriting mult_matrix_vector_numpy_row_oriented.py


In [6]:
%%bash
sudo perf stat -S -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_vector_numpy_row_oriented.py


 Performance counter stats for 'python3 mult_matrix_vector_numpy_row_oriented.py' (2 runs):

      240065888790      cycles                                                        ( +-  0.31% )
      596676621081      instructions              #    2.49  insn per cycle           ( +-  0.28% )
         549617732      cache-references                                              ( +-  0.84% )
          87432788      cache-misses              #   15.908 % of all cache refs      ( +-  0.29% )

      59.657694617 seconds time elapsed                                          ( +-  0.32% )



In [7]:
%%bash
sudo perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_vector_numpy_row_oriented.py


 Performance counter stats for 'system wide' (2 runs):

S0-C0           2         2022241353      cycles                                                      
S0-C0           2          749567422      instructions              #    0.37  insn per cycle         
S0-C0           2            5565741      cache-references                                            
S0-C0           2            1418960      cache-misses              #   25.495 % of all cache refs    
S0-C1           2          978643966      cycles                                                      
S0-C1           2          468796681      instructions              #    0.48  insn per cycle         
S0-C1           2            5124516      cache-references                                            
S0-C1           2            2519200      cache-misses              #   49.160 % of all cache refs    
S0-C2           2         1827829093      cycles                                                      
S0-C2           

# Nivel 3 de BLAS

Otras comparaciones que podemos realizar entre los tiempos de ejecución, *cache-references* y *cache-misses* utilizando la versión vectorizada y no vectorizada, es con el algoritmo de multiplicación de matrices $C = C + AB$ con $A \in \mathbb{R}^{m \times r}, B \in \mathbb{R}^{r \times n}, C \in \mathbb{R}^{m \times n}$. Este algoritmo se cataloga como de nivel 3 de BLAS pues realiza una **cantidad de trabajo cúbica** sobre una **cantidad cuadrática de datos**. Ver [level 3](http://www.netlib.org/blas/#_level_3) para más ejemplos de algoritmos en el álgebra lineal en esta categoría.

## Multiplicación de matrices

En LAPACK encontramos al algoritmo de multiplicación matricial con nombres como [sgemm](http://www.netlib.org/lapack/explore-html/d4/de2/sgemm_8f.html), [dgemm](http://www.netlib.org/lapack/explore-html/d7/d2b/dgemm_8f.html), [cgemm](http://www.netlib.org/lapack/explore-html/d6/d5b/cgemm_8f.html) y [zgemm](http://www.netlib.org/lapack/explore-html/d7/d76/zgemm_8f.html) para los casos de precisión simple, doble o números complejos respectivamente. 

El algoritmo de multiplicación de matrices puede escribirse en diferentes versiones. Por ejemplo la versión  **i,j,k** es:

```
for i in range(m):
    for j in range(n):
        for k in range(r):
            C[i][j]+=A[i][k]*B[k][j]
    
```

y la versión **j,k,i** es:

```
for j in range(n):
    for k in range(r):
        for i in range(m):
            C[i][j]+=A[i][k]*B[k][j]
```

**Comentarios:**

* Cualquiera de las versiones (hay $3!$ de ellas) involucra una cantidad de trabajo del orden $\mathcal{O}(mnr)$, la cual es cúbica. Es posible interpretar ésta cantidad de trabajo con una frase del tipo "si duplicamos cada dimensión de $A$ entonces la cantidad de trabajo se incrementa por un factor de $8$".

* Distintas versiones tienen distinto patrón de acceso a los datos de $A, B$ y $C$. Por ejemplo, para la variante **i,j,k** el **inner loop** realiza un producto punto que requiere acceso a renglones de $A$ y columnas de $B$ en una forma izquierda-derecha y abajo-arriba respectivamente. La variante **j,k,i** involucra una operación *saxpy* en el **inner loop** y acceso por columnas a la matriz $C$ y a $A$. Un resúmen de lo anterior se presenta en el siguiente cuadro:

<img src="https://dl.dropboxusercontent.com/s/rw81crmo1b7fjvb/tabla_con_versiones_mult_matrices.png?dl=0" heigth="450" width="450">


### Comparación entre versiones vectorizadas y no vectorizadas

Consideramos matrices $A, B \in \mathbb{R}^{10^4 \times 10^4}$ con entradas pseudoaleatorias. **No se sugiere ejecutar los siguientes ejemplos para máquinas que tengan menos de 8gb de memoria. Si se ejecutó el ejemplo de *gaxpy* de nivel 2 de BLAS, no es necesario volver a ejecutar la siguiente celda para crear a la matriz A**

In [21]:
np.random.seed(2020)
m=10**4
r=10**4

A=np.random.rand(m,r)
fileA='A.txt'
np.savetxt(fileA,A)

In [44]:
np.random.seed(2021)
r=10**4
n=10**4

B=np.random.rand(r,n)
fileB='B.txt'
np.savetxt(fileB,B)

In [45]:
%%file mult_matrix_matrix.py
m=10**4
r=10**4
n=10**4

A = []
B = []
fileA='A.txt'
fileB='B.txt'

with open(fileA,'r') as f:
    for l in f:
        A.append([float(k) for k in l.replace('\n','').replace(' ',',').split(',')]) 
        
with open(fileB,'r') as f:
    for l in f:
        B.append([float(k) for k in l.replace('\n','').replace(' ',',').split(',')])  

C=[[0]*n for i in range(m)]

for i in range(m):
    for j in range(n):
        for k in range(r):
            C[i][j]+=A[i][k]*B[k][j]


Writing mult_matrix_matrix.py


In [46]:
%%file mult_matrix_matrix_numpy.py
import numpy as np
m=10**4
r=10**4
n=10**4

fileA='A.txt'
fileB='B.txt'
A = np.loadtxt(fileA)
B = np.loadtxt(fileB)
C = A@B

Writing mult_matrix_matrix_numpy.py


In [47]:
%%bash
sudo perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 1 python3 mult_matrix_matrix.py

Process is terminated.


... 36 min y todavía no terminaba... cambiar a r 1

In [1]:
%%bash
sudo perf stat -S -a -e cycles,instructions,cache-references,cache-misses -r 1 python3 mult_matrix_matrix.py

Process is terminated.


In [3]:
%%bash
sudo perf stat -S -a -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_matrix_numpy.py


 Performance counter stats for 'system wide' (2 runs):

      836044127523      cycles                                                        ( +-  0.57% )
     1686463020251      instructions              #    2.02  insn per cycle           ( +-  0.43% )
        4086628234      cache-references                                              ( +-  1.72% )
         882062956      cache-misses              #   21.584 % of all cache refs      ( +-  4.13% )

     130.609360500 seconds time elapsed                                          ( +-  0.66% )



In [2]:
%%bash
sudo perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 2 python3 mult_matrix_matrix_numpy.py


 Performance counter stats for 'system wide' (2 runs):

S0-C0           2        89616044167      cycles                                                      
S0-C0           2       121446222500      instructions              #    1.36  insn per cycle         
S0-C0           2          811435068      cache-references                                            
S0-C0           2          194368638      cache-misses              #   23.954 % of all cache refs    
S0-C1           2        88898510819      cycles                                                      
S0-C1           2       121389910135      instructions              #    1.37  insn per cycle         
S0-C1           2          722839867      cache-references                                            
S0-C1           2          162558289      cache-misses              #   22.489 % of all cache refs    
S0-C2           2       563600069053      cycles                                                      
S0-C2           

### Otras versiones de la multiplicación de matrices apoyándose de las formas (i,j,k), (j,k,i),...

Si utilizamos la versión **i,j,k** podemos reescribir la multiplicación de matrices en una forma:

```
for i in range(m):
    for j in range(n)
        C[i][j]+= A[i,:].dot(B[:,j])
```

y hemos vectorizado el *inner loop* para usar productos punto que es una operación nivel 1 de BLAS.

Además `numpy` nos provee de funcionalidad para reescribir ésta forma de productos punto en una como sigue:


```
for i in range(m):
    C[i,:]+= A[i,:]@B
```

**Obs**: obsérvese que en esta reescritura en el loop se realizan $m$ operaciones *gaxpy* de nivel 2. Esta versión como se verá a continuación es más rápida que la que utiliza operaciones de nivel 1 de BLAS:

#### Comparación de versiones de multiplicación de matrices utilizando nivel 1 vs nivel 2 de BLAS

**Ejemplo**

Utilizamos matrices $A, B \in \mathbb{R}^{10^3 \times 10^3}$ pseudoaleatorias:

In [19]:
np.random.seed(2020)
m=10**3
r=10**3

A=np.random.rand(m,r)
fileA_10_3='A_10_3.txt'
np.savetxt(fileA_10_3,A)

In [20]:
np.random.seed(2021)
m=10**3
r=10**3

B=np.random.rand(m,r)
fileB_10_3='B_10_3.txt'
np.savetxt(fileB_10_3,B)

In [23]:
%%file mult_matrix_matrix_numpy_dot_product.py
import numpy as np
m=10**3
n=10**3

fileA_10_3='A_10_3.txt'
fileB_10_3='B_10_3.txt'
A = np.loadtxt(fileA_10_3)
B = np.loadtxt(fileB_10_3)
C = np.zeros((m,n))
for i in np.arange(m):
        for j in np.arange(n):
                C[i][j]+= A[i,:].dot(B[:,j])

Writing mult_matrix_matrix_numpy_dot_product.py


In [26]:
%%bash
sudo perf stat -S -a -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix_numpy_dot_product.py


 Performance counter stats for 'system wide' (5 runs):

       17727880724      cycles                                                        ( +-  2.67% )
       31196800089      instructions              #    1.76  insn per cycle           ( +-  0.18% )
         174120561      cache-references                                              ( +-  1.57% )
          14645042      cache-misses              #    8.411 % of all cache refs      ( +-  8.98% )

       3.817488998 seconds time elapsed                                          ( +-  0.55% )



In [27]:
%%bash
sudo perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix_numpy_dot_product.py


 Performance counter stats for 'system wide' (5 runs):

S0-C0           2          582438322      cycles                                                      
S0-C0           2          225885082      instructions              #    0.39  insn per cycle         
S0-C0           2             274724      cache-references                                            
S0-C0           2             130031      cache-misses              #   47.332 % of all cache refs    
S0-C1           2          642407408      cycles                                                      
S0-C1           2          272043994      instructions              #    0.42  insn per cycle         
S0-C1           2             546749      cache-references                                            
S0-C1           2             324188      cache-misses              #   59.294 % of all cache refs    
S0-C2           2          596216226      cycles                                                      
S0-C2           

In [30]:
%%file mult_matrix_matrix_numpy_dot_product_gaxpy.py
import numpy as np
m=10**3
n=10**3
fileA_10_3='A_10_3.txt'
fileB_10_3='B_10_3.txt'
A = np.loadtxt(fileA_10_3)
B = np.loadtxt(fileB_10_3)
C = np.zeros((m,n))
for i in np.arange(m):
    C[i,:] = A[i,:]@B

Overwriting mult_matrix_matrix_numpy_dot_product_gaxpy.py


In [31]:
%%bash
sudo perf stat -S -a -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix_numpy_dot_product_gaxpy.py


 Performance counter stats for 'system wide' (5 runs):

       11010075589      cycles                                                        ( +-  0.66% )
       14922627515      instructions              #    1.36  insn per cycle           ( +-  0.26% )
         142059490      cache-references                                              ( +-  1.24% )
           8572872      cache-misses              #    6.035 % of all cache refs      ( +- 11.05% )

       1.455285799 seconds time elapsed                                          ( +-  0.44% )



In [32]:
%%bash
sudo perf stat -S -a --per-core -e cycles,instructions,cache-references,cache-misses -r 5 python3 mult_matrix_matrix_numpy_dot_product_gaxpy.py


 Performance counter stats for 'system wide' (5 runs):

S0-C0           2         1448177294      cycles                                                      
S0-C0           2          623468047      instructions              #    0.43  insn per cycle         
S0-C0           2           29581790      cache-references                                            
S0-C0           2             575180      cache-misses              #    1.944 % of all cache refs    
S0-C1           2         1477272782      cycles                                                      
S0-C1           2          656096587      instructions              #    0.44  insn per cycle         
S0-C1           2           29881953      cache-references                                            
S0-C1           2             608210      cache-misses              #    2.035 % of all cache refs    
S0-C2           2         1457641798      cycles                                                      
S0-C2           

**Comentarios:**

* Obsérvese que en las implementaciones anteriores ambas versiones utilizan la vectorización mediante `numpy`.

* A continuación se muestra una tabla que presenta el número de *flops* realizados por distintas operaciones del álgebra lineal:

<img src="https://dl.dropboxusercontent.com/s/rvqkokicaqkwrif/tabla_con_flops_para_operaciones_alg_lineal.png?dl=0" heigth="550" width="550">

* Como se observa en la tabla anterior, *gaxpy* (nivel 2 de BLAS) realiza más *flops* que un producto punto. Aún así, los tiempos medidos con `perf` indican que la versión de *gaxpy* es más rápida que la versión con productos punto (nivel 1 de BLAS) para la multiplicación de matrices ya que  aprovecha mejor el *data reuse* y *data locality*. Esto se logra pues *gaxpy* disminuye el número de *cache misses* y por tanto el tráfico hacia y desde el caché, el llamado *data movement/motion*. 


# Blocking algorithms para multiplicación de matrices

El *data reuse* y el *data locality* como se vio en el ejemplo pasado se incrementa al utilizar operaciones de nivel BLAS mayores. También disminuye el *data motion* hacia y desde el caché lo que disminuye el tráfico de datos o *data movement/motion* pues se atiende uno de los cuellos de botella que se revisó en [2.1.Un_poco_de_historia_y_generalidades](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.1.Un_poco_de_historia_y_generalidades.ipynb) conocido con el nombre de *bottleneck* de Von Neumann. 

Por lo anterior hay algoritmos que se diseñan con el objetivo de aprovechar lo más posible el *data reuse* y el *data locality* y se nombran *blocking algorithms*. Entre los *blocking algorithms* encontramos a los que trabajan con matrices por bloques pues son más eficientes al utilizar un nivel $3$ de BLAS. Una matriz $A \in \mathbb{R}^{m \times n}$ por bloques se puede escribir como:

<img src="https://dl.dropboxusercontent.com/s/s77weq74yoi2rfc/matrix_A_by_blocks.png?dl=0" heigth="300" width="300">

donde: $m_1 + \dots + m_q = m$, $n_1 + \dots + n_r = n$. Con esta definición se llama a la matriz $A$, una matriz por bloques de tamaño $q \times r$.

**Comentarios:** 

* Hay que tener en cuenta que trabajar con matrices por bloques utiliza mayor cantidad de memoria que un algoritmo que opera a un nivel escalar esto puede ser benéfico o no dependiendo del problema, máquina o arquitectura en la que se esté trabajando.

* Trabajar con *blocking algorithms* permite simplificar notación matemática. Su escritura ayuda a pensar en una implementación con cómputo en paralelo.

## Algunas operaciones en matrices por bloques

Entre las operaciones que son posibles realizar a un nivel por bloques se encuentran:

### Multiplicación por escalares

$$\begin{array}{l}
\mu \left[ \begin{array}{cc}
A_{11} & A_{12}\\
A_{21} & A_{22}\\
A_{31} & A_{32}
\end{array}
\right] = 
\left[\begin{array}{cc}
\mu A_{11} & \mu A_{12}\\
\mu A_{21} & \mu A_{22}\\
\mu A_{31} & \mu A_{32}
\end{array}
\right] 
\end{array}
$$

con $\mu \in \mathbb{R}$.

### Transposición

$$\begin{array}{l}
\left[ \begin{array}{cc}
A_{11} & A_{12}\\
A_{21} & A_{22}\\
A_{31} & A_{32}
\end{array}
\right]^T = 
\left[\begin{array}{ccc}
A_{11}^T & A_{21}^T & A_{31}^T\\
A_{12}^T & A_{22}^T & A_{32}^T\\
\end{array}
\right] 
\end{array}
$$

### Suma

$$\begin{array}{l}
\left[ \begin{array}{cc}
A_{11} & A_{12}\\
A_{21} & A_{22}\\
A_{31} & A_{32}
\end{array}
\right] + 
\left[\begin{array}{cc}
B_{11} & B_{12}\\
B_{21} & B_{22}\\
B_{31} & B_{32}
\end{array}
\right] =
\left[\begin{array}{cc}
A_{11} + B_{11} & A_{12} + B_{12}\\
A_{21} + B_{21} & A_{22} + B_{22}\\
A_{31} + B_{31} & A_{32} + B_{32}
\end{array}
\right]
\end{array}
$$

### Multiplicación

$$\begin{array}{l}
\left[ \begin{array}{cc}
A_{11} & A_{12}\\
A_{21} & A_{22}\\
A_{31} & A_{32}
\end{array}
\right] \cdot
\left[\begin{array}{cc}
B_{11} & B_{12}\\
B_{21} & B_{22}\\
\end{array}
\right] =
\left[\begin{array}{cc}
A_{11}B_{11} + A_{12}B_{21} & A_{11}B_{12} + A_{12}B_{22}\\
A_{21}B_{11} + A_{22}B_{21} & A_{21}B_{12} + A_{22}B_{22}\\
A_{31}B_{11} + A_{32}B_{21} & A_{31}B_{12} + A_{32}B_{22}
\end{array}
\right]
\end{array}
$$

**Obs:** el número de columnas de los bloques $A_{11}, A_{21}, A_{31}$ deben coincidir con el número de renglones de $B_{11}, B_{12}$. Así como con los bloques $A_{12}, A_{22}, A_{32}$ y $B_{21}, B_{22}$.

Considérese que se particionan a las matrices $A, B, C$ como sigue:

<img src="https://dl.dropboxusercontent.com/s/j0p89i19vf5r88k/A_B_C_matrices_by_blocks.png?dl=0" heigth="400" width="400">


Esto es, $A,B,C$ son $N \times N$ matrices por bloques con $\ell \times \ell$ bloques. Entonces, para índices $\alpha = 1,2,\dots, N$ y $\beta = 1,2,\dots,N$ se tiene que el bloque $\alpha \beta$ de $C$ se obtiene: $C_{\alpha \beta} = \displaystyle \sum_{\gamma=1}^N A_{\alpha \gamma} B_{\gamma \beta}$. 

El algoritmo es:

```
for alpha in np.arange(N)
    i = np.arange((alpha - 1)*l + 1,alpha*l + 1) #hay que revisar si están bien definidos los índices
    for beta in np.arange(N)
        j = np.arange((beta - 1)*l + 1, beta*l + 1) #hay que revisar si están bien definidos los índices
        for gamma in np.arange(N)
            k = np.arange((gamma - 1)*l + 1, gamma*l + 1) #hay que revisar si están bien los índices
            C[i][:,j]+= A[i][:,k]*B[k][:,j]

```